In [2]:
!pip install "sagemaker>=2.48.0" --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 535.0/535.0 KB 7.3 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.98.0-py2.py3-none-any.whl size=748029 sha256=7ff6261c1a9287fb7a9f19c1558609f3f2b6d3c55b679b69a2290857d9c61460
  Stored in directory: /root/.cache/pip/wheels/6d/01/8d/9b0d73afa29b591c959018e5975e0edd8a2f1d4c7c59b15451
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.97.0
    Uninstalling sagemaker-2.97.0:
      Successfully uninstalled sagemaker-2.97.0
You should consider upgrading via the '/opt/conda/bin/python -m pip install --upgrade pip' command.


In [42]:
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker 

role = sagemaker.get_execution_role()

# Hub Model configuration. https://huggingface.co/models
hub = {'HF_MODEL_ID':'alvaroalon2/biobert_genetic_ner','HF_TASK':'token-classification', 'aggregation_strategy':'simple'}

# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
   env=hub,
   role=role, # iam role with permissions to create an Endpoint
   transformers_version="4.17.0", # transformers version used
   pytorch_version="1.10.2", # pytorch version used
   py_version="py38", # python version of the DLC
)

In [43]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
   initial_instance_count=1,
   instance_type="ml.m5.xlarge"
)

-----!

In [140]:
detect_genetic_entities({"inputs": "HER2 Gene is known to cause breast cancer. A2M gene is not known for breast cancer"})

HER2 Gene
A2M gene


In [126]:
# # delete endpoint
# predictor.delete_endpoint()

# Textract

In [142]:


def start_job(client, s3_bucket_name, object_name):
    response = None
    response = client.start_document_text_detection(
        DocumentLocation={
            'S3Object': {
                'Bucket': s3_bucket_name,
                'Name': object_name
            }})

    return response["JobId"]


def is_job_complete(client, job_id):
    time.sleep(1)
    response = client.get_document_text_detection(JobId=job_id)
    status = response["JobStatus"]
    print("Job status: {}".format(status))

    while(status == "IN_PROGRESS"):
        time.sleep(1)
        response = client.get_document_text_detection(JobId=job_id)
        status = response["JobStatus"]
        print("Job status: {}".format(status))

    return status


def get_job_results(client, job_id):
    pages = []
    time.sleep(1)
    response = client.get_document_text_detection(JobId=job_id)
    pages.append(response)
    print("Resultset page received: {}".format(len(pages)))
    next_token = None
    if 'NextToken' in response:
        next_token = response['NextToken']

    while next_token:
        time.sleep(1)
        response = client.\
            get_document_text_detection(JobId=job_id, NextToken=next_token)
        pages.append(response)
        print("Resultset page received: {}".format(len(pages)))
        next_token = None
        if 'NextToken' in response:
            next_token = response['NextToken']

    return pages

In [131]:
import boto3
import time
from IPython.display import Image, display
s3_bucket_name = "aws-experiments-b18961"
document_name = "7/ER0197 ER-AC0F Clinical Report_Redacted_Sanitized.pdf"
region = "us-east-1"
client = boto3.client('textract', region_name=region)

In [133]:
job_id = start_job(client, s3_bucket_name, document_name)
print("Started job with id: {}".format(job_id))
if is_job_complete(client, job_id):
    response = get_job_results(client, job_id)

Started job with id: fdd72de8f0533719c0ad8f7bff51b39d3dc14a2a06265205ab085aa59fcd2c07
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: IN_PROGRESS
Job status: SUCCEEDED
Resultset page received: 1
Resultset page received: 2
Resultset page received: 3


In [218]:
def detect_clinical_entities(data):
    entities =  comprehend_medical.detect_entities(Text=data)
    for entity in entities["Entities"]:
        print("- {}".format(entity["Text"]))
        print ("   Type: {}".format(entity["Type"]))
        print ("   Category: {}".format(entity["Category"]))
        if(entity["Traits"]):
            print("   Traits:")
            for trait in entity["Traits"]:
                print ("    - {}".format(trait["Name"]))
        print("\n")

def detect_genetic_entities(data):
    sentence=data['inputs']
    result=predictor.predict(data)
    i=0
    start=[]
    end=[]
    for item in result:
        if result[i]['entity']=='B-GENETIC':
            start.append(result[i]['start'])
            if result[i]['index']>1:
                end.append(result[i-1]['end'])
        i=i+1
    if len(result)>0:
        if len(start)>len(end):
            end.append(result[(len(result))-1]['end'])

    for i in range(len(end)):
        print("- {}".format(sentence[start[i]:end[i]]))
        print ("   Type: {}".format("GENETIC"))
        

In [219]:
# Print detected text
#text=""
comprehend_medical = boto3.client('comprehendmedical')

print("*****ENTITY EXTRACTION REPORT*****")

for result_page in response:
    for item in result_page["Blocks"]:
        if item["BlockType"] == "LINE":
            detect_clinical_entities(item["Text"])
            detect_genetic_entities({"inputs":item["Text"]})

*****ENTITY EXTRACTION REPORT*****
- LEFT
   Type: DIRECTION
   Category: ANATOMY


- BREAST
   Type: SYSTEM_ORGAN_SITE
   Category: ANATOMY


- Estrogen Receptor
   Type: GENETIC
- Progesterone Receptor
   Type: GENETIC
- Her2neu
   Type: GENETIC
- immunoperoxidase panel
   Type: TEST_NAME
   Category: TEST_TREATMENT_PROCEDURE


- immunoperoxidase
   Type: GENETIC
- cleared
   Type: DX_NAME
   Category: MEDICAL_CONDITION
   Traits:
    - NEGATION


- U.S.
   Type: ADDRESS
   Category: PROTECTED_HEALTH_INFORMATION


- 1988
   Type: DATE
   Category: PROTECTED_HEALTH_INFORMATION


- laboratory testing
   Type: TEST_NAME
   Category: TEST_TREATMENT_PROCEDURE


- Monoclonal antibodies
   Type: TEST_NAME
   Category: TEST_TREATMENT_PROCEDURE


- PgR636
   Type: TEST_NAME
   Category: TEST_TREATMENT_PROCEDURE


- SP1, and PgR636
   Type: GENETIC
- 
   Type: GENETIC
- LSAB immunohistochemical assay
   Type: TEST_NAME
   Category: TEST_TREATMENT_PROCEDURE


- estrogen
   Type: TEST_NAME
   Ca